In [1]:
import pandas
from sklearn import preprocessing
import numpy as np

In [2]:
num_segments = 19
test_split = 0.3

In [3]:
data = pandas.read_csv('/home/carsten/workspaces/isws/models/predictions_cnn_19k.csv')
num_test_songs = int(len(data)*test_split)

In [4]:
data = data.drop(['offset', 'duration'],axis=1)
data.head()

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
0,reggae.00009.wav,0.421017,1679.434736,2024.534270,3466.256905,0.064117,-262.811999,126.180913,11.227370,21.112270,...,0.007860,9.522888e-08,0.006322,0.002053,0.031482,0.000387,0.000067,0.007997,0.942741,0.001092
1,reggae.00009.wav,0.398483,1875.148780,2227.413557,4031.587601,0.069351,-220.923940,117.441621,4.900961,20.067881,...,0.010861,4.979003e-08,0.003688,0.000187,0.026418,0.000958,0.000008,0.003163,0.954332,0.000385
2,reggae.00009.wav,0.339041,1833.160181,2189.649126,3916.098976,0.067055,-206.509028,117.687803,3.399634,28.708057,...,0.068167,3.036692e-05,0.033795,0.002462,0.011202,0.018917,0.000151,0.005191,0.853980,0.006105
3,reggae.00009.wav,0.344620,1592.273248,2028.807317,3360.189056,0.054104,-232.590603,127.788885,7.052849,31.868343,...,0.041583,5.319529e-06,0.020122,0.005269,0.010508,0.001775,0.000155,0.008868,0.906942,0.004772
4,reggae.00009.wav,0.378558,1501.627471,1940.269736,3026.676750,0.051956,-243.499728,132.915085,4.320031,20.461475,...,0.007871,8.119686e-08,0.004065,0.003663,0.019545,0.000149,0.000071,0.009919,0.953424,0.001293


In [5]:
for i in range(len(data.filename)):
    res = data.iat[i,0]
    res = res.split('.')[0]
    data.iat[i,0] = res

In [6]:
genre_list = data.iloc[:, 0]
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(genre_list)

In [7]:
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, 1:], dtype = float))
np.array(data.iloc[:, 1:])

array([[4.21017349e-01, 1.67943474e+03, 2.02453427e+03, ...,
        7.99694100e-03, 9.42740980e-01, 1.09187390e-03],
       [3.98482721e-01, 1.87514878e+03, 2.22741356e+03, ...,
        3.16288320e-03, 9.54331930e-01, 3.85428870e-04],
       [3.39040828e-01, 1.83316018e+03, 2.18964913e+03, ...,
        5.19050100e-03, 8.53980240e-01, 6.10466390e-03],
       ...,
       [5.82671040e-01, 2.81592156e+03, 2.20142615e+03, ...,
        3.23582070e-03, 1.03490170e-04, 2.04184890e-01],
       [5.84403050e-01, 2.82644200e+03, 2.19657691e+03, ...,
        3.43380590e-03, 1.91280710e-04, 1.78307820e-01],
       [5.70103128e-01, 2.86434960e+03, 2.20523546e+03, ...,
        4.54706840e-03, 3.20502000e-04, 1.87299640e-01]])

In [8]:
X_test = X[:num_test_songs]
X_train = X[num_test_songs:]
y_test = y[:num_test_songs]
y_train = y[num_test_songs:]

In [9]:
from keras import models
from keras import layers

from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [10]:
val_split = int(19000*0.2)
x_val = X_train[:val_split]
partial_x_train = X_train[val_split:]

y_val = y_train[:val_split]
partial_y_train = y_train[val_split:]
x_norm = preprocessing.scale(x_val)

In [11]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                 ModelCheckpoint(filepath='./../../../models/ann-model-best.h5', monitor='val_loss', save_best_only=True)]

model.fit(partial_x_train,
          partial_y_train,
          epochs=200,
          batch_size=512,
          callbacks=callbacks,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 9500 samples, validate on 3800 samples
Epoch 1/200
9500/9500 [==============================] - 1s 88us/step - loss: 1.1746 - acc: 0.6980 - val_loss: 0.5185 - val_acc: 0.8434
Epoch 2/200
9500/9500 [==============================] - 0s 48us/step - loss: 0.3648 - acc: 0.8861 - val_loss: 0.5289 - val_acc: 0.8411
Epoch 3/200
9500/9500 [==============================] - 0s 45us/step - loss: 0.2566 - acc: 0.9173 - val_loss: 0.4982 - val_acc: 0.8368
Epoch 4/200
9500/9500 [==============================] - 0s 44us/step - loss: 0.2036 - acc: 0.9339 - val_loss: 0.4955 - val_acc: 0.8363
Epoch 5/200
9500/9500 [==============================] - 0s 48us/step - loss: 0.1674 - acc: 0.9443 - val_loss: 0.5261 - val_acc: 0.8411
Epoch 6/200
9500/9500 [==============================] - 0s 47us/step - loss: 0.1487 - acc: 0.9522 - val_loss: 0.5988 - val_acc: 0.8274
Epoch 7/200
9500

In [12]:
results

[1.6380022586208949, 0.6950877192982456]

In [16]:
p = model.predict(X_test)

In [17]:
a = 0
for i in range(len(p)):
    if int(np.argmax(p[i])) == int(y_test[i]):
        a = a + 1
print(a/num_test_songs)

0.6950877192982456
